In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import dask

from dask.distributed import Client
from datetime import datetime

client = Client(n_workers=3)

# CADES configuration
# from dask_jobqueue import SLURMCluster
# cluster = SLURMCluster(project='covid19', queue='covid19', cores=30, memory='300 GB', job_extra=["-N 1"], interface="ib0")
# client = Client(cluster)

In [ ]:
def to_date(d):
    if ":" == d[-3:-2]:
        d = d[:-3]+d[-2:]
    return d

In [ ]:
# poi = dd.read_csv("/lustre/or-hydra/cades-birthright/nickrobison/mobility-analysis/data/reference/core-poi-join/2020/**/core_poi*.csv")
poi = dd.read_csv("/Users/raac/Development/covid/core-poi-join/2020/06/**/core_poi*.csv")
poi = poi[['safegraph_place_id', 'latitude', 'longitude']]

In [ ]:
poi = poi.drop_duplicates(['safegraph_place_id'])

In [ ]:
def compute_location_cbg(df):
    import geopandas as gpd
    from shapely.geometry import Point
    
    localdf = df[['latitude', 'longitude']]
    
    cbgs = gpd.read_file("/Users/raac/Development/covid/mobility-analysis/data/reference/census/block_groups.shp")
    cbgs.crs = 'epsg:4269'
    cbgs = cbgs.to_crs('epsg:4326')
    local_gdf = gpd.GeoDataFrame(localdf, crs="epsg:4326",\
                                       geometry=[Point(xy) for xy in \
                                                zip(localdf['longitude'], localdf['latitude'])])
    local_gdf = gpd.sjoin(local_gdf, cbgs, how='left', op='within')
    
    return local_gdf.GEOID.rename('location_cbg')
poi['location_cbg'] = poi.map_partitions(compute_location_cbg, meta=('location_cbg', 'str'))

In [ ]:
poi.to_parquet("data/reference/Joined2.parquet")